This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Generative deep learning

## Text generation

### A brief history of generative deep learning for sequence generation

### How do you generate sequence data?

### The importance of the sampling strategy

**Reweighting a probability distribution to a different temperature**

In [1]:
import numpy as np
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

### Implementing text generation with Keras

#### Preparing the data

**Downloading and uncompressing the IMDB movie reviews dataset**

**Creating a dataset from text files (one file = one sample)**

In [2]:
import tensorflow as tf
from tensorflow import keras
dataset = keras.utils.text_dataset_from_directory(
    directory="../data/aclImdb", label_mode=None, batch_size=256)
# 删除许多评论中都有的HTML标签<br />
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))

Found 50006 files belonging to 1 classes.


**Preparing a `TextVectorization` layer**

In [3]:
from tensorflow.keras.layers import TextVectorization

sequence_length = 100
# 只考虑前15000个最常见的单词，其他单词都被视为未登录词元[UNK]
vocab_size = 15000
# 返回单词整数索引的序列
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(dataset)

**Setting up a language modeling dataset**

In [4]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    # 删除序列中最后一个单词来创建输入
    x = vectorized_sequences[:, :-1]
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

#### A Transformer-based sequence-to-sequence model

In [5]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

**A simple Transformer-based language model**

In [6]:
from tensorflow.keras import layers
embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

### A text-generation callback with variable-temperature sampling

**The text-generation callback**

In [7]:
import numpy as np

# 将单词索引映射为字符串，用于文本编码
tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

def sample_next(predictions, temperature=1.0):
    # 从概率分布进行采样，温度可变
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,
                 generate_length,
                 model_input_length,
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        # 要生成的单词个数
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        # 用于采样的温度
        self.temperatures = temperatures
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            # 生成文本时，初始文本为提示词
            sentence = self.prompt
            for i in range(self.generate_length):
                # 将当前序列输入模型
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                # 采样一个新词
                next_token = sample_next(predictions[0, i, :])
                sampled_token = tokens_index[next_token]
                # 将这个新词添加到当前序列中，并重复上述过程
                sentence += " " + sampled_token
            print(sentence)

prompt = "This movie"
# 使用不同温度值对文本进行采样，以展示温度对文本生成的影响
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

**Fitting the language model**

In [8]:
model.fit(lm_dataset, epochs=200, callbacks=[text_gen_callback])

Epoch 1/200
196/196 [==============================] - ETA: 0s - loss: 5.5917== Generating with temperature 0.2
This movie [UNK] once is again that if thats an really terrorists did when alan you finds have himself a in sandy his william fashion destroying every the station little ever pays in over the a state chilling [UNK] is was terrible [UNK] and direction it this [UNK] film 2000 is of
== Generating with temperature 0.5
This movie is should as good the were age disturbing you but have its no wants 2 to a ask key the [UNK] movie the youll great play as yourself much look bad at music the peace opening i terrible was actors far made more those from acting other was than happening
== Generating with temperature 0.7
This movie film was knows a going man on who tv was that harsh is her following and oliver im when a i high dont to think variety i development was of 14 what year was i a like few a puerto song day a yes loving its life first to comments
== Generating with temperature 1.0


196/196 [==============================] - ETA: 0s - loss: 4.4469== Generating with temperature 0.2
This movie movie started absolutely where horrible the looking cgi bad left flat them so out this not was exactly the what acting this had really or there could was make easily this one film tension is or not profound as like possible a the few best is scenes this from movie
== Generating with temperature 0.5
This movie movies is is hilarious laughable you and cant dont get watch to your come walking across is very the young most [UNK] enjoy serial danny horror spring movie small no screen matter the im word surprised vampire to chronicles refer my to mind any the while main watching in this
== Generating with temperature 0.7
This movie film looks very like much it more however so i bad have movies seen so and many many things overrated to writers start in a my full own ben life stiller is and hilarious his sitting first here time its when a fun big moment stupid actually about i any
== 

This movie film had is a a testament jungle to on her death role penalty as in gabriel 1940 which unfortunately had lacks no a conscience great about montage the and actors spring to break insult their real exquisite love setting it in was whiskey their polanski inherent clearly in echoes films
196/196 [==============================] - 43s 220ms/step - loss: 4.1759
Epoch 11/200
196/196 [==============================] - ETA: 0s - loss: 4.1246== Generating with temperature 0.2
This movie film is from not the a native very mafia predictable film movie noir where directed the by double don standards murray from turned good into black three and portions white of gun time here were is made excellent in films his clint usual eastwood still spaghetti endearing westerns performance with
== Generating with temperature 0.5
This movie is is classic story your about hunchback suicide kids with and his a great medical [UNK] teacher [UNK] teaching his a students wealthy not kid by the his space par

This movie is is a not good this movie is made because so of many everything lowbudget is it actually does quite deliver suck this let is me the start concept out slowly liveaction despite the death original scenes cartoon of improve needles this hurts happens the to brains idol and getgo
== Generating with temperature 1.5
This movie movie has made females me the watch plot it because runs it it destroys is itself a friendly sentimental place entertaining near and the fits finale the of character the overall bette charm meets error the animals found life with style her and as to well provoke as her soon
196/196 [==============================] - 52s 262ms/step - loss: 3.9574
Epoch 16/200
196/196 [==============================] - ETA: 0s - loss: 3.9222== Generating with temperature 0.2
This movie was is awful crap as if usual it i although would its be more cheap clueless the i stupidity wont of give any anything characters away basically from these the actors plotline you that hate ha

This movie movie has is all one kung of fu those movies rare during fail songs ones abstract as hrs backgrounds of arent [UNK] real when emotion [UNK] you around see them this experiencing movie almost far anyone more checking to out understand the the flaws review that does comes not very
== Generating with temperature 1.0
This movie remember plays i as actually a and kid it i just usually couldnt love stand which up was as an old american teen gi but jane he is usually escape a from wallace home hectic [UNK] car expertise chases to recruits green robert makeup mitchum to at play the slain
== Generating with temperature 1.5
This movie film changes entered the whilst form the of slaughtered a and serial gratuitous killer astonishing inside torture thinking in i the might biggest have seven seen minutes only before seven but minutes was the a english genius version that of is the good descent guy credited at is least simply
196/196 [==============================] - 44s 225ms/step - los

...

Epoch 180/200
196/196 [==============================] - ETA: 0s - loss: 2.8915== Generating with temperature 0.2
This movie is is absolutely funny beyond obnoxious a and few somewhat sort [UNK] of moments a that fine will actors give like some this of film the  film is very very real youre in for its dated fans of innocent women may be shocked by the casual selling intent but
== Generating with temperature 0.5
This movie movie had was its pretty moments good and acting its it like was theres promising a humorous bit story about the reign [UNK] of immature fire gullible has grows badly [UNK] grinch but who where loves first to scenes drop with your the sensibility guy suffering does from not [UNK]
== Generating with temperature 0.7
This movie is was genuinely unique abysmal it hardly was gets jungle away molly from kim something novak special did wonder we if know this steve the [UNK] difference is between a vertigo stranger a is good it guy will gets now the he suspense chases althoug

This movie mess has is nothing pathetic good i script have chucky ever is done one a of friend the chucky cat movie dolls tiffany for and the chucky back movie indelible doll [UNK] for beautiful the sweet ring and and magic voodoo etc [UNK] even magic when but flash after premiered
196/196 [==============================] - 46s 230ms/step - loss: 2.8850
Epoch 185/200
196/196 [==============================] - ETA: 0s - loss: 2.8836== Generating with temperature 0.2
This movie is was an a excellent special example movie of that lush is the elizabeth pace minute of performance production streisand has plays wonderful a chemistry addict between that her is emotionally almost cunning interesting and like [UNK] at a times [UNK] [UNK] of stage draw actors the goes whole to
== Generating with temperature 0.5
This movie film is lasted awesome so over long the done scary dead parts alive but and sad alive movie and sticky the moving jerk work ship to image embark i on was the really disappointi

This movie movie is is a never [UNK] got or to changed [UNK] anything eyes at [UNK] [UNK] most is people released are in my love mind but and just it think is of not schools going go to to but school also still [UNK] [UNK] girls as although a their child
== Generating with temperature 1.5
This movie frog is [UNK] coming animated first superman affair tries if to you bridge have the way story home never was dies burnt in significantly an speaking attempt [UNK] to with fly his on death pure he [UNK] looks around more on like this turner terrible had female to voices have
196/196 [==============================] - 46s 231ms/step - loss: 2.8771
Epoch 190/200
196/196 [==============================] - ETA: 0s - loss: 2.8759== Generating with temperature 0.2
This movie was is excellent about i two loved men it and is a difficult terrific movie movie about from some the straight traveling actors [UNK] below along the and lines the kept actors the help scenery of very the good ultimate content

This movie movie just gets isnt [UNK] a more picture genuine than [UNK] usual could cliché there the been movie many this people well there while is no acting romance imagine the how one [UNK] felt are felt young real whos people willing are to either support not mexican iraq either ok
== Generating with temperature 1.0
This movie movie im takes going it in into which the its skin a of very walking bad tall effects fool are around probably if the i guys had were 9 just or 80 25 of not you that would good watch if another jacks laughs nerd or like hell my will
== Generating with temperature 1.5
This movie movie is moved the me pace from did beginning not to as leave long you if can you old can sentiment call improve it its this kind movie of is way not better possible not charlie your sheen story is becomes based a on graphic life novel and by also
196/196 [==============================] - 45s 230ms/step - loss: 2.8698
Epoch 195/200
196/196 [==============================] - ETA: 0s - 

This movie is is rated mainly so about highly three rated people actors that not have only any star expectations yet this there is is also the trash 12 that theres attract a viewers talented who people is people going on through really many bad things movies look like like people a
== Generating with temperature 0.7
This movie little based [UNK] on on the not novel a single great way story is line original has and been alter padded ego your of brother aside lets from have julie pimp [UNK] the no colonel rifle who is was listed raised as a the tough sexual scene forces where to
== Generating with temperature 1.0
This movie is is half absolutely set false up and as you if could you stop thought wasting not money speaking my vehicles personalities and than talking almost [UNK] every the script characters is are artificial erotic that tyrone are power out line of whoever have invited it me you to knew
== Generating with temperature 1.5
This movie caper minimal was passionate part channel a

### Wrapping up